In [1]:
import torch
import torch.nn.functional as F
import torchvision.transforms as transforms
from PIL import Image
import numpy as np

In [2]:
import os
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import models, transforms
from PIL import Image
from sklearn.metrics import roc_auc_score, precision_recall_fscore_support, accuracy_score

import lightning as L
from lightning import Trainer
from lightning.pytorch.callbacks import ModelCheckpoint, EarlyStopping, BackboneFinetuning
from lightning.pytorch.loggers import MLFlowLogger

import mlflow
from model_def import LightningCheXpertModel

In [3]:
import mlflow
from mlflow.tracking import MlflowClient

# We don't have to set MLflow tracking URI because we set it in an environment variable
mlflow.set_tracking_uri("http://129.114.26.91:8000/") 

client = MlflowClient()
experiment = client.get_experiment_by_name("chexpert-classifier")
experiment

<Experiment: artifact_location='mlflow-artifacts:/2', creation_time=1746807822515, experiment_id='2', last_update_time=1746807822515, lifecycle_stage='active', name='chexpert-classifier', tags={}>

In [4]:
runs = client.search_runs(experiment_ids=[experiment.experiment_id], 
    order_by=["metrics.val_accuracy DESC"], 
    max_results=2)

In [5]:
best_run = runs[0]  # The first run is the best due to sorting
best_run_id = best_run.info.run_id
best_test_accuracy = best_run.data.metrics["val_accuracy"]
model_uri = f"runs:/{best_run_id}/model"

print(f"Best Run ID: {best_run_id}")
print(f"Test Accuracy: {best_test_accuracy}")
print(f"Model URI: {model_uri}")

Best Run ID: 8fa6d6af16584563951285b9fbfcf7c7
Test Accuracy: 0.8118082284927368
Model URI: runs:/8fa6d6af16584563951285b9fbfcf7c7/model


In [6]:
def preprocess_image(img):
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])

    return transform(img).unsqueeze(0)

In [7]:
# class LightningCheXpertModel(L.LightningModule):
#         def __init__(self, config, num_classes=14):
#             super().__init__()
#             self.save_hyperparameters()
#             self.model = models.densenet121(weights=models.DenseNet121_Weights.DEFAULT)
#             in_features = self.model.classifier.in_features
#             self.model.classifier = nn.Linear(in_features, num_classes)
#             self.criterion = nn.BCEWithLogitsLoss()
#             self.lr = config["lr"]

#         @property
#         def backbone(self):
#             return self.model.features

#         def forward(self, x):
#             return self.model(x)

#         def training_step(self, batch, batch_idx):
#             x, y = batch
#             logits = self(x)
#             mask = (y == 0.0) | (y == 1.0)
#             loss = self.criterion(logits[mask], y[mask])
#             self.log("train_loss", loss, prog_bar=True, sync_dist=True)
#             return loss

#         def validation_step(self, batch, batch_idx):
#             x, y = batch
#             logits = self(x)
#             probs = torch.sigmoid(logits)
#             mask = (y == 0.0) | (y == 1.0)
#             y_true = y[mask].detach().cpu().numpy()
#             y_pred = (probs[mask] > 0.5).float().detach().cpu().numpy()

#             loss = self.criterion(logits[mask], y[mask])
#             precision, recall, f1, _ = precision_recall_fscore_support(y_true, y_pred, average='macro', zero_division=0)
#             acc = accuracy_score(y_true, y_pred)
#             try:
#                 auroc = roc_auc_score(y_true, probs[mask].detach().cpu().numpy(), average='macro')
#             except:
#                 auroc = 0.0

#             self.log("val_loss", loss, prog_bar=True, sync_dist=True)
#             self.log("val_precision", precision, prog_bar=True, sync_dist=True)
#             self.log("val_recall", recall, prog_bar=True, sync_dist=True)
#             self.log("val_f1", f1, prog_bar=True, sync_dist=True)
#             self.log("val_accuracy", acc, prog_bar=True, sync_dist=True)
#             self.log("val_auroc", auroc, prog_bar=True, sync_dist=True)
#             return loss

#         def configure_optimizers(self):
#             return optim.Adam(self.model.classifier.parameters(), lr=self.lr)


In [8]:
model = mlflow.pytorch.load_model(model_uri)

Exception ignored in: <function tqdm.__del__ at 0x79d6f45e2280>
Traceback (most recent call last):
  File "/opt/conda/lib/python3.9/site-packages/tqdm/std.py", line 1147, in __del__
    self.close()
  File "/opt/conda/lib/python3.9/site-packages/tqdm/notebook.py", line 286, in close
    self.disp(bar_style='danger', check_delay=False)
AttributeError: 'tqdm' object has no attribute 'disp'
2025/05/10 18:53:20 WARNING mlflow.pytorch: Stored model version '2.7.0+cu126' does not match installed PyTorch version '2.5.1+cpu'


In [9]:
import os
import io

In [10]:
print(os.getcwd())

/home/jovyan


In [11]:
def get_model_size_in_mb(model) -> float:
    buffer = io.BytesIO()
    torch.save(model.state_dict(), buffer)
    size_mb = buffer.getbuffer().nbytes / (1024 ** 2)
    return size_mb

# Example usage:
size = get_model_size_in_mb(model)
print(f"Model size: {size:.2f} MB")

Model size: 27.15 MB


In [15]:
save_dir = r"./"
save_path = os.path.join(save_dir, "state_dict.pt")
os.makedirs(save_dir, exist_ok=True)
torch.save(model.state_dict(), save_path)

In [13]:
print(model.__class__)
print(model.__class__.__module__)

<class '__main__.LightningCheXpertModel'>
__main__


In [16]:
config = {
    "initial_epochs": 1,
    "total_epochs": 2,
    "patience": 5,
    "batch_size": 64,
    "lr": 1e-4,
    "fine_tune_lr": 1e-6,
}

In [19]:
from model_def import LightningCheXpertModel
model = LightningCheXpertModel(config)
model.load_state_dict(torch.load(r"./state_dict.pt"))



/tmp/ipykernel_310/4279995210.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(r"./state_dict.pt"))


<All keys matched successfully>

In [21]:
model.eval()

# 4. Save full model
os.makedirs("saved_model", exist_ok=True)
torch.save(model, "saved_model/mlflowModel1.pt")

In [ ]:
model = torch.load("saved_model/mlflowModel1.pt")

# 2. Set it to evaluation mode for inference
model.eval()

image_path = "test_image.jpeg"
image = Image.open(image_path).convert("RGB")
image_tensor = preprocess_image(image)

# 3. Use it for inference
with torch.no_grad():
    output = model(image_tensor)